# <span style="color:teal">RBFE Network - Analysis</span>


In [ ]:
# import libraries

from scipy.stats import sem as sem
import sys
import glob

if "/home/anna/Documents/cinnabar" not in sys.path:
    sys.path.insert(1, "/home/anna/Documents/cinnabar")
import cinnabar

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

In [ ]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "tyk2"
main_dir = f"{bench_folder}/extracted/{protein}"
# main_dir = f"/backup/42_reruns/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_combined.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"

# if os.path.exists(f"{main_dir}/outputs_extracted/results"):
#     results_folder = f"{main_dir}/outputs_extracted/results"
# elif os.path.exists(f"{main_dir}/outputs/results"):
#     results_folder = f"{main_dir}/outputs/results"
# else:
#     raise ValueError(
#         f"results directory not found in the {main_dir}. please make sure results were written using the analysis script previously in the pipeline"
#     )
results_folder = f"{main_dir}/outputs_extracted/results"
output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)

In [ ]:
all_analysis_object = analysis_network(
    results_folder,
    exp_file=exp_file,
    net_file=net_file,
    output_folder=output_folder,
    analysis_prot=ana_file,
    # method = "updated",
    engines = "SOMD"
)

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute_results()

In [ ]:
all_analysis_object.sort_ligands_by_binding_affinity(engine="SOMD")

In [ ]:
all_analysis_object.sort_ligands_by_experimental_binding_affinity()

In [ ]:
# check if there are any failed perturbations
failed_perts = all_analysis_object.failed_runs("AMBER")
# print(failed_perts)
for pert in failed_perts:
    print(pert)

In [ ]:
# can compute convergence for all
all_analysis_object.compute_convergence(main_dir=main_dir)
all_analysis_object.plot_convergence()

In [ ]:
# MAE calculate
for eng in all_analysis_object.engines:
    mae = all_analysis_object._stats_object.compute_mue(y=eng, pert_val="pert")
    stats_string = f"MAE: {mae[0]:.2f} +/- {mae[1]:.2f} kcal/mol"
    all_analysis_object.plot_scatter_pert(
        engine=eng,
        **{"title": stats_string, "key": False, "save": f"/home/anna/Desktop/{eng}"},
    )

There are different options for plotting:
bar (pert or lig)
scatter (pert or lig) - can be plotted using cinnabar
eng vs eng
other results (eg from other workflows)
outliers
histograms (for the error of the runs, sem, legs)

In [ ]:
# bar
all_analysis_object.plot_bar_lig()
all_analysis_object.plot_bar_pert()

# scatter
all_analysis_object.plot_scatter_lig()
all_analysis_object.plot_scatter_pert()
all_analysis_object.plot_scatter_lig(use_cinnabar=True)
all_analysis_object.plot_scatter_pert(use_cinnabar=True)

for eng in all_analysis_object.engines:
    all_analysis_object.plot_scatter_lig(engine=eng)
    all_analysis_object.plot_scatter_pert(engine=eng)

    # outliers
    all_analysis_object.plot_outliers(engine=eng)
    all_analysis_object.plot_outliers(engine=eng, pert_val="val")

all_analysis_object.plot_histogram_legs()
all_analysis_object.plot_histogram_repeats()
all_analysis_object.plot_histogram_sem(pert_val="pert")
all_analysis_object.plot_histogram_sem(pert_val="val")

calculate the stats:

In [ ]:
all_analysis_object.calc_mad_engines(pert_val="pert")
all_analysis_object.calc_mad_engines(pert_val="val")
stats_dict = all_analysis_object.calc_stats()

plot other results

In [ ]:
perts, ligs = get_info_network(
    "/backup/42_reruns/tyk2/execution_model/network_combined_reruns_ejm42.dat"
)

for eng in all_analysis_object.engines:
    other_results = glob.glob(
        f"/backup/42_reruns/tyk2/outputs_extracted/results/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    all_analysis_object.compute_other_results(other_results, name=f"{eng}_reruns")
    # all_analysis_object.plot_other_results(
    #     name=f"{eng}_reruns", engine=eng, pert_val="pert"
    # )
    all_analysis_object.plot_bar_pert(
        engine=[eng, f"{eng}_reruns"], **{"values": perts}
    )
    res1 = all_analysis_object._stats_object.compute_mue("pert", y=eng)
    res2 = all_analysis_object._stats_object.compute_mue("pert", y=f"{eng}_reruns")
    res3 = all_analysis_object._stats_object.compute_mue(
        "pert", y=eng, x=f"{eng}_reruns"
    )
    print(
        f"og vs exp is {res1[0]}, reruns vs exp is {res2[0]}, and compared to each other theya re {res3[0]}"
    )

all_analysis_object.plot_bar_pert(
    engine=all_analysis_object.engines + all_analysis_object.other_results_names,
    **{"values": perts},
)

In [ ]:
! cat /home/anna/Documents/pmx_test/gromacs_23/me/execution_model/protocol_gromacs20.dat

# ! cat /home/anna/Documents/pmx_test/gromacs_23/me/execution_model/config19.dat

In [ ]:
eng = "GROMACS"

for name in ["ambersc3","ambersc","scamber2"]:

    other_results = glob.glob(
        f"{results_folder}/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    # bound_results = glob.glob(
    #     f"{results_folder}/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    # )
    # free_results = glob.glob(
    #     f"{results_folder}/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    # )
    all_analysis_object.compute_other_results(
        other_results, name=f"{name}", method=f"{name}", use_cinnabar=False, bound_files=bound_results, free_files=free_results
    )


In [ ]:
all_analysis_object.change_name("GROMACS","alpha=0.3 sigma=0.25")
all_analysis_object.change_name("ambersc3","alpha=0.4 sigma=0.25")
all_analysis_object.change_name("ambersc","alpha=0.5 sigma=0.25")
all_analysis_object.change_name("scamber2","alpha=0.5 sigma=0.30")


In [ ]:
all_analysis_object.plot_bar_pert(engine=all_analysis_object.other_results_names)

In [ ]:
# for eng in all_analysis_object.engines:

    for name in ["pymbar3","pymbar4"]:

        other_results = glob.glob(
            f"/backup/42_reruns/tyk2/outputs_extracted/results_{name}/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
        )
        bound_results = glob.glob(
            f"/backup/42_reruns/tyk2/outputs_extracted/results_{name}/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
        )
        free_results = glob.glob(
            f"/backup/42_reruns/tyk2/outputs_extracted/results_{name}/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
        )
        all_analysis_object.compute_other_results(
            other_results, name=f"{name}_{eng}", method=None, use_cinnabar=False, bound_files=bound_results, free_files=free_results
        )

# perts, ligs = get_info_network("/backup/AMBER_reruns/tyk2/execution_model/network.dat")
# eng = "AMBER"
# other_results = glob.glob(
#     f"/home/anna/Documents/benchmark/extracted/tyk2/outputs_extracted/results/final_summary_AMBER_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
# )
# name="previous"
# all_analysis_object.compute_other_results(other_results, name=f"{name}", method=f"None", use_cinnabar=False)



In [ ]:
for eng in all_analysis_object.engines:

    other_results = glob.glob(
        f"{bench_folder}/extracted/{protein}/outputs_extracted/results/final_summary_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    bound_results = glob.glob(
        f"{bench_folder}/extracted/{protein}/outputs_extracted/results/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    free_results = glob.glob(
        f"{bench_folder}/extracted/{protein}/outputs_extracted/results/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    all_analysis_object.compute_other_results(
        other_results, name=f"prev_{eng}", method=None, use_cinnabar=False, bound_files=bound_results, free_files=free_results
    )

In [ ]:
all_analysis_object._stats_object.compute_mue(pert_val="free",y="SOMD",x="prev_SOMD")

In [ ]:
all_analysis_object.plot_bar_pert(engine=["GROMACS","prev_GROMACS"])

In [ ]:
all_analysis_object.plot_bar_leg(leg="free", engine=["SOMD","prev_SOMD"])

In [ ]:
# all_analysis_object.change_name("GROMACS","plain")
all_analysis_object.plot_bar_pert(
    engine=all_analysis_object.other_results_names,
    **{"yerror": True},
)  # , **{"values":perts}
# all_analysis_object.plot_outliers(**{"yerror": True})

plot convergence

In [ ]:
ex_outputs_folder = f"{main_dir}/outputs_extracted"

converg_obj = plot_convergence(
    ex_outputs_folder,
    perturbations=all_analysis_object.perturbations,
    engines=all_analysis_object.engines,
    file_ext=file_ext,
)

converg_obj.plot_convergence_all()

remove a perturbation

In [ ]:
# check which perturbations are outliers
all_analysis_object.plot_outliers(engine="GROMACS", outliers=5, pert_val="pert")
# all_analysis_object._plotting_object.outlier(engines="AMBER", outliers=5, name="Exscientia")

In [ ]:
# remove perturbation for all engines
all_analysis_object.remove_perturbations("lig_c~lig_d")

remove outliers

In [ ]:
all_analysis_object.remove_outliers(threshold=10, verbose=True)

calculate only the experimental results for some of the perturbations

In [ ]:
protein = "mcl1"

bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
# net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
# exp_file = f"{bench_folder}/inputs/experimental/tyk2.yml"
exp_file = f"/home/anna/Documents/benchmark/inputs/experimental/{protein}.yml"

output_folder = validate.folder_path(f"{main_dir}/diff_methods_analysis", create=True)
results_folder = f"{main_dir}/diff_methods_analysis/results"

all_analysis_object = analysis_network(exp_file=exp_file)

all_analysis_object.perturbations = [
    "lig_27~lig_59",
    "lig_27~lig_48",
    "lig_28~lig_47",
    "lig_35~lig_52",
    "lig_38~lig_48",
    "lig_43~lig_45",
    "lig_53~lig_56",
    "lig_30~lig_40",
    "lig_27~lig_48",
]
all_analysis_object.ligands = get_ligands_from_perts(all_analysis_object.perturbations)
all_analysis_object.get_experimental()
all_analysis_object.get_experimental_pert()

plot only a small amount of perturbations to compare different methods

In [ ]:
perturbations = ["lig_30~lig_40", "lig_27~lig_48"]

In [ ]:
protein = "mcl1"

bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
# net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
# exp_file = f"{bench_folder}/inputs/experimental/tyk2.yml"
exp_file = f"/home/anna/Documents/benchmark/inputs/experimental/{protein}.yml"

output_folder = validate.folder_path(f"{main_dir}/diff_methods_analysis", create=True)
results_folder = f"{main_dir}/diff_methods_analysis/results"

all_analysis_object = analysis_network(
    results_folder,
    exp_file=exp_file,
    engines="AMBER",
    #    net_file=net_file,
    output_folder=output_folder,
    analysis_ext=ana_file,
)

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute(cycle_closure=False, use_cinnabar=False)

In [ ]:
other_names = ["exscientia_inputs", "both", "Exscientia"]

for name in other_names:
    all_analysis_object.compute_other_results(
        sorted(glob.glob(f"{results_folder}/other/*{name}*.csv")),
        name=name,
        use_cinnabar=False,
    )

all_analysis_object._plotting_object.set_colours(
    {"exscientia_inputs": "red", "both": "yellow", "Exscientia": "purple"}
)

In [ ]:
all_analysis_object._plotting_object.scatter(
    pert_val="pert", engines=["exscientia_inputs"], name="Exscientia"
)

compare to the fwf analysis

In [ ]:
# choose an engine
eng = "SOMD"

# first need to add the fwf path
all_analysis_object._add_fwf_path(
    "/home/anna/Documents/september_2022_workshops/freenrgworkflows/networkanalysis"
)

# get the experimental dictionary
exp_dicts = all_analysis_object._get_exp_fwf()
for key in exp_dicts[0]:
    print(f"{key} : {exp_dicts[0][key][0]}, {exp_dicts[0][key][1]}")

# get the network analysis
fwf_dict = all_analysis_object._get_ana_fwf(engine=eng)
for key in fwf_dict:
    print(f"{key} : {fwf_dict[key][0]}, {fwf_dict[key][1]}")

# get fwf stats
fwf_stats = all_analysis_object._get_stats_fwf(engine=eng)

plotting just fwf data per ligand

In [ ]:
dict_y = fwf_dict
dict_exp = exp_dicts[0]

df1 = plotting_engines.match_dicts_to_df(dict_exp, dict_y, "experimental", "fwf")
df1

df1.plot.bar(
    y=["freenrg_fwf", "freenrg_experimental"],
    yerr=df1[["err_fwf", "err_experimental"]].T.values,
    title=f"fwf, experimental, {eng}",
    xlabel="ligands",
    ylabel="dG (kcal/mol)",
)

df1.dropna()
df1.plot.scatter(
    x="freenrg_experimental",
    y="freenrg_fwf",
    xerr="err_experimental",
    yerr="err_fwf",
    title=f"fwf, {eng}",
    xlabel="experimental dG (kcal/mol)",
    ylabel="fwf dG (kcal/mol)",
)

# calculating using the cinnabar stats
f_mae = all_analysis_object._stats_object._compute_stats(
    x=df1["freenrg_experimental"],
    y=df1["freenrg_fwf"],
    xerr=df1["err_experimental"],
    yerr=df1["err_fwf"],
    statistic="MUE",
)
print(f_mae)